<a href="https://colab.research.google.com/github/deboraeverlyab/bi-data-analyst-challenge/blob/main/relatorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Relatório Teste Analista de BI/Dados

Relatório dividio em 4 partes:
1. Preparação de dados
2. SQL para análises
3. Capacidade Analítica
4. Integrações

## PARTE 1 - Projeto: Preparação de Dados do Google Sheets

Esta parte tem como objetivo realizar a extração, limpeza e exportação de dados provenientes de uma planilha do Google Sheets.

Os principais passos realizados são:

1. Autenticação com a conta do Google e acesso à planilha via API do Google Sheets.
2. Extração dos dados em formato bruto, onde cada linha da planilha contém todos os campos agrupados em uma única célula separada por `;`.
3. Processamento e reorganização dos dados para estrutura tabular, com validação da consistência das colunas.
4. Tratamento dos dados, incluindo:
   - Conversão de colunas de **data**, tanto em formato serial (Excel) quanto textual.
   - Conversão de colunas numéricas:
     - **Números inteiros** (como "impressões" e "cliques").
     - **Números decimais** com separador `,` (como "valor gasto" e "leads").
5. Exportação dos dados tratados para um arquivo `.csv`, pronto para uso em análises/dashboards.
6. Envio dos dados tratados de volta para o Google Sheets, criando uma aba separada chamada "Dados Tratados" dentro da mesma planilha original.

O fluxo foi desenvolvido para lidar com planilhas que armazenam os dados no estilo "CSV em célula", exigindo um pré-processamento manual linha a linha.

Planilha usada: https://docs.google.com/spreadsheets/d/1YytDjZsIXBTg0ceoXoKq9C8-BMPNOKopZbywCuZSzRY/edit?usp=sharing

1. Instalação e importação de bibliotecas

In [46]:
# Instala a biblioteca para acesso ao Google Sheets
!pip install gspread

# Bibliotecas para manipulação de dados e autenticação com o Google
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import set_with_dataframe

2. Autenticação com a conta do Google

In [47]:
# Autenticando no ambiente do Google Colab com sua conta
auth.authenticate_user()

# Obtém as credenciais da conta conectada
creds, _ = default()

# Autoriza o acesso à API do Google Sheets com as credenciais
gc = gspread.authorize(creds)

3. Acessar a planilha do Google Sheets

In [48]:
# ID da planilha
spreadsheet_id = '1YytDjZsIXBTg0ceoXoKq9C8-BMPNOKopZbywCuZSzRY'

# Acessa a primeira aba da planilha
try:
    worksheet = gc.open_by_key(spreadsheet_id).sheet1
except Exception as e:
    print(f"Erro ao abrir a planilha: {e}")

4. Obter dados da planilha

In [49]:
# Pega todos os valores da planilha
raw_data = worksheet.get_all_values()

5. Separar cabeçalho e preparar os dados

Esse trecho limpa e organiza cada linha dos dados brutos vindos do Google Sheets, porque os dados foram colados ou exportados para a planilha com tudo junto em uma única célula, ou seja, cada linha da planilha tem todos os campos concatenados em uma única string, separados por ;.

In [50]:
# Pega o cabeçalho (em formato de string única)
header_string = raw_data[0][0]
header = header_string.split(';')

header

['canal',
 'campanha',
 'grupo de anúncio',
 'data',
 'impressões',
 'cliques',
 'leads',
 'valor gasto']

In [51]:
# Inicializa uma lista para armazenar os dados tratados
processed_data = []

# Processa as linhas (excluindo o cabeçalho)
for row in raw_data[1:]:
    if row and row[0]:
        row_string = row[0].strip()
        if row_string.startswith('"') and row_string.endswith('"'):
            row_string = row_string[1:-1]
        parts = row_string.split(';')
        if len(parts) == len(header):
            processed_data.append(parts)
        else:
            print(f"Aviso: Linha com colunas inconsistentes: {row_string}")
    else:
        print(f"Aviso: Linha vazia ou mal formatada: {row}")

# Imprime apenas as 3 primeiras linhas processadas
for linha in processed_data[:3]:
    print(linha)

['facebook', 'atr_leads | Genérica  + Geral Promo [CBO]', 'LAL 1% COMPRADORES', '45473', '1899', '10', '1', '48,3']
['facebook', 'atr_leads | avalie o seu usado [CBO] 2', 'LAL 1% COMPRADORES — AJUSTADO', '45473', '3687', '38', '4', '60,47']
['facebook', 'atr_leads | promo - creta [CBO] 2', 'RMKT', '45473', '1080', '7', '3', '32,62']


6. Criar o DataFrame com os dados tratados

In [52]:
df = pd.DataFrame(processed_data, columns=header)

df

,canal,campanha,grupo de anúncio,data,impressões,cliques,leads,valor gasto
0,facebook,atr_leads | Genérica + Geral Promo [CBO],LAL 1% COMPRADORES,45473,1899,10,1,"48,3"
1,facebook,atr_leads | avalie o seu usado [CBO] 2,LAL 1% COMPRADORES — AJUSTADO,45473,3687,38,4,"60,47"
2,facebook,atr_leads | promo - creta [CBO] 2,RMKT,45473,1080,7,3,"32,62"
3,facebook,atr_leads | promo - hb20 (CBO) 2,LAL Compradores 1%,45473,1059,3,1,"13,96"
4,facebook,atr_leads | promo - hb20 (CBO) 2,RMKT,45473,768,5,,"16,77"
...,...,...,...,...,...,...,...,...
862,google,atr_search | Blumenau,HB20,45535,88,14,3,"19,78"
863,google,atr_search | Jaraguá do Sul,HB20,45535,129,10,3,"20,71"
864,google,atr_search | Joinville,HB20,45535,145,9,1,"17,64"
865,google,atr_search | marca,HB20,45535,339,29,1,"25,65"


7. Conversão da coluna de datas

In [53]:
# Tenta converter 'data' de número serial (formato Excel) para datetime
try:
    df['data'] = pd.to_datetime(pd.to_numeric(df['data']), unit='D', origin='1899-12-30')
except (ValueError, KeyError):
    print("Coluna 'data' não está em formato serial. Tentando como string...")
    df['data'] = pd.to_datetime(df['data'], errors='coerce')
except Exception as e:
    print(f"Erro ao converter 'data': {e}")

df['data']

,data
0,2024-06-30
1,2024-06-30
2,2024-06-30
3,2024-06-30
4,2024-06-30
...,...
862,2024-08-31
863,2024-08-31
864,2024-08-31
865,2024-08-31


8. Conversão de colunas numéricas

In [54]:
# Converte colunas numéricas inteiras
for col in ['impressões', 'cliques']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    else:
        print(f"Coluna '{col}' não encontrada.")

# Converte colunas com casas decimais (trocando ',' por '.')
for col in ['leads', 'valor gasto']:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce').fillna(0)
    else:
        print(f"Coluna '{col}' não encontrada.")

df.head()

,canal,campanha,grupo de anúncio,data,impressões,cliques,leads,valor gasto
0,facebook,atr_leads | Genérica + Geral Promo [CBO],LAL 1% COMPRADORES,2024-06-30,1899.0,10.0,1.0,48.30
1,facebook,atr_leads | avalie o seu usado [CBO] 2,LAL 1% COMPRADORES — AJUSTADO,2024-06-30,3687.0,38.0,4.0,60.47
2,facebook,atr_leads | promo - creta [CBO] 2,RMKT,2024-06-30,1080.0,7.0,3.0,32.62
3,facebook,atr_leads | promo - hb20 (CBO) 2,LAL Compradores 1%,2024-06-30,1059.0,3.0,1.0,13.96
4,facebook,atr_leads | promo - hb20 (CBO) 2,RMKT,2024-06-30,768.0,5.0,0.0,16.77


9. Exportar o DataFrame como CSV

In [55]:
output_csv_file = "planilha_processada.csv"
df.to_csv(output_csv_file, sep=';', index=False)

print(f"✅ Dados processados salvos em: {output_csv_file}")

✅ Dados processados salvos em: planilha_processada.csv


10. Envia pra aba na planilha do Google para análise no Looker

In [56]:
# Abre a planilha completa (não só a aba)
spreadsheet = gc.open_by_key(spreadsheet_id)

# Tenta acessar a aba 'Dados Tratados'. Se não existir, cria.
try:
    worksheet_tratada = spreadsheet.worksheet("Dados Tratados")
    worksheet_tratada.clear()  # Limpa conteúdo se já existir
except:
    worksheet_tratada = spreadsheet.add_worksheet(title="Dados Tratados", rows="1", cols="1")

# Envia o DataFrame tratado para essa aba
set_with_dataframe(worksheet_tratada, df)

print("✅ Dados tratados enviados para a aba 'Dados Tratados' na planilha.")

✅ Dados tratados enviados para a aba 'Dados Tratados' na planilha.


Observações:

O dashboard no Google Looker Studio foi preparado com a estrutura necessária, incluindo:

- Principais campanhas e grupos de anúncios

- Indicadores calculados (CPM, CTR, CPL, Taxa de Conversão)

- Filtros por data e outros critérios

- Tabelas com capacidade de drill-down

Dashboard: https://lookerstudio.google.com/reporting/b9e32f36-519e-41f1-84de-5196bddc4713

## PARTE 2 - SQL para análises

Esta etapa, os dados previamente tratados foram armazenados em um banco de dados local utilizando SQLite, com o objetivo de facilitar consultas analíticas por meio de comandos SQL.

O processo incluiu:

1. Criação do banco local (campanhas.db) e definição da tabela campanhas, com as seguintes colunas:

- data (DATE)

-  campanha (TEXT)

- impressoes (INTEGER)

- cliques (INTEGER)

- leads (INTEGER)

- valor_gasto (REAL)

2. Inserção dos dados tratados diretamente do DataFrame para a tabela usando pandas.to_sql.

3. Execução de duas consultas para responder às perguntas-chave:

- Qual campanha trouxe mais leads no período total?
→ Consulta utilizando SUM(leads) agrupado por campanha.

- Qual campanha apresentou o melhor CPL (Custo por Lead)?
→ Cálculo com SUM(valor_gasto) / SUM(leads) ordenado do menor para o maior valor.

As respostas foram obtidas diretamente via SQL, e o banco pode ser reutilizado para novas análises futuras.

1. Importação de bibliotecas

In [57]:
import sqlite3

2. Criar banco de dados local

In [58]:
# Conectar/criar banco local
conn = sqlite3.connect('campanhas.db')
cursor = conn.cursor()

3. Criação e população da tabela

In [59]:
# Criação da tabela
cursor.execute('''
CREATE TABLE IF NOT EXISTS campanhas (
    data DATE,
    campanha TEXT,
    impressoes INTEGER,
    cliques INTEGER,
    leads INTEGER,
    valor_gasto REAL
)
''')

conn.commit()

# Inserir dados do DataFrame no banco
df_renamed = df.rename(columns={
    'data': 'data',
    'campanha': 'campanha',
    'impressões': 'impressoes',
    'cliques': 'cliques',
    'leads': 'leads',
    'valor gasto': 'valor_gasto'
})
df_renamed.to_sql('campanhas', conn, if_exists='replace', index=False)

867

###Pergunta 1: Qual campanha trouxe mais leads no período total?

In [60]:
query1 = """
SELECT campanha, SUM(leads) AS total_leads
FROM campanhas
GROUP BY campanha
ORDER BY total_leads
DESC
LIMIT 1
"""

cursor.execute(query1)
result1 = cursor.fetchone()
print(f"Campanha com mais leads: {result1[0]} com {result1[1]} leads")

Campanha com mais leads: atr_leads | promo - hb20 (CBO) 2 com 493.0 leads


###Pergunta 2: Qual campanha tem melhor CPL (Custo por Lead) no período total?

CPL = valor_gasto / leads

In [61]:
query2 = """
SELECT campanha,
       SUM(valor_gasto) / NULLIF(SUM(leads), 0) AS cpl
FROM campanhas
GROUP BY campanha
HAVING SUM(leads) > 0
ORDER BY cpl ASC
LIMIT 1
"""

cursor.execute(query2)
result2 = cursor.fetchone()
print(f"Campanha com melhor CPL: {result2[0]} com CPL de R$ {result2[1]:.2f}")

Campanha com melhor CPL: atr_leads | promo - hb20 (CBO) 2 com CPL de R$ 11.34


4. Schema da tabela

In [62]:
cursor.execute("PRAGMA table_info(campanhas)")
schema = cursor.fetchall()
print("Schema da tabela 'campanhas':")
for col in schema:
    print(col)

Schema da tabela 'campanhas':
(0, 'canal', 'TEXT', 0, None, 0)
(1, 'campanha', 'TEXT', 0, None, 0)
(2, 'grupo de anúncio', 'TEXT', 0, None, 0)
(3, 'data', 'TIMESTAMP', 0, None, 0)
(4, 'impressoes', 'REAL', 0, None, 0)
(5, 'cliques', 'REAL', 0, None, 0)
(6, 'leads', 'REAL', 0, None, 0)
(7, 'valor_gasto', 'REAL', 0, None, 0)


Consultas Extras

In [63]:
# Query para calcular o total de leads por campanha e a razão percentual
query3 = """
SELECT
    campanha,
    SUM(leads) AS total_leads,
    ROUND(100.0 * SUM(leads) / (SELECT SUM(leads) FROM campanhas), 2) AS percentual
FROM campanhas
GROUP BY campanha
ORDER BY total_leads ASC
LIMIT 5
"""

cursor.execute(query3)
results = cursor.fetchall()

print("Leads por campanha (ordenado do menor para o maior):")
for campanha, total_leads, percentual in results:
    print(f"{campanha}: {total_leads} leads ({percentual}%)")

print("......")

# Query para calcular CPL
query4 = """
SELECT campanha,
       SUM(valor_gasto) / NULLIF(SUM(leads), 0) AS cpl
FROM campanhas
GROUP BY campanha
HAVING SUM(leads) > 0
ORDER BY cpl DESC
LIMIT 5
"""

cursor.execute(query4)
results = cursor.fetchall()

print("CPL por campanha (ordenado do maior para o menor):")
for campanha, CPL in results:
    print(f"{campanha}: R${CPL:.2f}")

Leads por campanha (ordenado do menor para o maior):
atr_leads | avalie o seu usado [CBO]: 2.0 leads (0.11%)
atr_search | joinviile (NEW) - TESTE: 3.0 leads (0.16%)
atr_search | Ponta Grossa: 6.0 leads (0.32%)
atr_search | AsU: 20.0 leads (1.06%)
atr_leads | Hyundai Day: 25.0 leads (1.32%)
......
CPL por campanha (ordenado do maior para o menor):
atr_search | joinviile (NEW) - TESTE: R$68.02
atr_leads | avalie o seu usado [CBO]: R$56.96
atr_search | Blumenau: R$55.49
atr_search | Joinville: R$31.66
atr_leads | Promo HR [CBO]: R$31.05


In [64]:
# Query para calcular o total de leads por canal e a razão percentual
query_canal_leads = """
SELECT
    canal,
    SUM(leads) AS total_leads,
    ROUND(100.0 * SUM(leads) / (SELECT SUM(leads) FROM campanhas), 2) AS percentual
FROM campanhas
GROUP BY canal
ORDER BY total_leads ASC
"""

cursor.execute(query_canal_leads)
results = cursor.fetchall()

print("Leads por canal (ordenado do menor para o maior):")
for canal, total_leads, percentual in results:
    print(f"{canal}: {total_leads} leads ({percentual}%)")

print("......")

# Query para calcular o CPL por canal
query_canal_cpl = """
SELECT
    canal,
    SUM(valor_gasto) / NULLIF(SUM(leads), 0) AS cpl
FROM campanhas
GROUP BY canal
HAVING SUM(leads) > 0
ORDER BY cpl DESC
"""

cursor.execute(query_canal_cpl)
results = cursor.fetchall()

print("CPL por canal (ordenado do maior para o menor):")
for canal, cpl in results:
    print(f"{canal}: R${cpl:.2f}")

Leads por canal (ordenado do menor para o maior):
google: 306.0 leads (16.19%)
facebook: 1584.0 leads (83.81%)
......
CPL por canal (ordenado do maior para o menor):
google: R$30.19
facebook: R$13.92


## PARTE 3 - Capacidade analítica

Principais Insights Identificados:

### Por Campanha:

1. Campanha de Alta Performance: A campanha atr_leads | promo - hb20 (CBO) 2 se destaca como líder tanto em volume de leads (493) quanto em eficiência de custo (CPL R$ 11,34)



2. Oportunidades de Otimização:
As campanhas mais caras que atrairam poucos leads foram:

- atr_leads | avalie o seu usado [CBO] com 2 leads e um investimento de R$ 56,96

- atr_search | joinviile (NEW) - TESTE com 3 leads e um investimento de R$ 68,02


- atr_search | Ponta Grossa com 6 leads e um investimento de R$ 55,49


3. Recomendações Estratégicas:

- Escalar campanhas eficientes: Aumentar investimento na campanha promo - HB20

- Otimizar campanhas de busca como revisar palavras-chave

- Melhorar segmentação: Focar em públicos com maior propensão à conversão

### Por Canal

1. Distribuição de Leads:

- Facebook foi o canal mais eficiente em volume de geração de leads, respondendo por 83,81% do total (1.584 leads).

- Google gerou apenas 16,19% dos leads (306 leads), uma diferença significativa em relação ao Facebook.

2. Custo por Lead (CPL):

- O CPL do Google está em R$30,19, mais que o dobro do CPL do Facebook (R$13,92).


3. Recomendações Estratégicas:

O canal Facebook tem um desempenho muito superior, tanto em volume de leads quanto em eficiência de custo. Isso indica que, embora o Google tenha gerado menos leads, ele também foi menos eficiente em termos de custo.

Pode ser interessante realocar parte do investimento do Google para o Facebook, ou investigar formas de otimizar as campanhas no Google para reduzir o CPL.

## PARTE 4 - Integrações

Consulta à API de Cotação do Dólar – Banco Central


Esta etapa realiza a extração das cotações do dólar comercial (compra e venda) dos últimos 7 dias úteis utilizando a API pública do Banco Central. Os dados são consultados no formato JSON, processados com Python (Pandas) e exportados em formato .csv para análise ou uso posterior.

<hr>

API - Navegador de Dados Banco Central

URL: https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/aplicacao#!/

Endereço padrão:
https ://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/[codigo_recurso]?$format=json&[Outros Parâmetros]

Os parâmetros disponíveis podem ser visualizados na documentação

Os códigos dos recursos disponibilizados são: Moedas, CotacaoDolarDia, CotacaoDolarPeriodo, CotacaoMoedaDia, CotacaoMoedaPeriodo

1. Instalação e importação de bibliotecas

In [65]:
# Instala a biblioteca para requisições WEB
!pip install gspread

# Bibliotecas para manipulação de dados e envio de requisições WEB
import requests
import pandas as pd
from datetime import datetime, timedelta

2. Definição do Período de Busca

In [66]:
data_fim = datetime.today()
data_inicio = data_fim - timedelta(days=7)

# Formatar as datas para o formato que a API espera (MM-DD-YYYY)
data_inicio_str = data_inicio.strftime('%m-%d-%Y')
data_fim_str = data_fim.strftime('%m-%d-%Y')

print(f"Período de busca definido: de {data_inicio.strftime('%d-%m-%Y')} a {data_fim.strftime('%d-%m-%Y')}")
print(f"Datas formatadas para a API: Início={data_inicio_str}, Fim={data_fim_str}")

Período de busca definido: de 15-07-2025 a 22-07-2025
Datas formatadas para a API: Início=07-15-2025, Fim=07-22-2025


3. Montagem da URL da API e Requisição

In [67]:
url = f"https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/" \
      f"CotacaoDolarPeriodo(dataInicial='{data_inicio_str}',dataFinalCotacao='{data_fim_str}')?" \
      f"$top=100&$format=json"

print(f"URL da API montada: {url}")

# Fazer a requisição HTTP para a API
response = requests.get(url)

# Trata a resposta
if response.status_code == 200:
    dados = response.json()['value']
    print(dados)
    df = pd.DataFrame(dados)
else:
  print(f"Erro ao acessar a API: {response.status_code}")

URL da API montada: https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial='07-15-2025',dataFinalCotacao='07-22-2025')?$top=100&$format=json
[{'cotacaoCompra': 5.5569, 'cotacaoVenda': 5.5576, 'dataHoraCotacao': '2025-07-15 13:04:30.654'}, {'cotacaoCompra': 5.5716, 'cotacaoVenda': 5.5722, 'dataHoraCotacao': '2025-07-16 13:08:40.079'}, {'cotacaoCompra': 5.5731, 'cotacaoVenda': 5.5737, 'dataHoraCotacao': '2025-07-17 13:02:31.315'}, {'cotacaoCompra': 5.546, 'cotacaoVenda': 5.5466, 'dataHoraCotacao': '2025-07-18 13:11:35.417'}, {'cotacaoCompra': 5.5619, 'cotacaoVenda': 5.5625, 'dataHoraCotacao': '2025-07-21 13:03:57.875'}]


4. Tratamento dos dados da requisição

In [68]:
# Limpa e reorganiza colunas principais
df = df[['dataHoraCotacao', 'cotacaoCompra', 'cotacaoVenda']]
df['dataHoraCotacao'] = pd.to_datetime(df['dataHoraCotacao']).dt.date
df = df.rename(columns={
    'dataHoraCotacao': 'Data',
    'cotacaoCompra': 'Cotação Compra',
    'cotacaoVenda': 'Cotação Venda'
})
df

,Data,Cotação Compra,Cotação Venda
0,2025-07-15,5.5569,5.5576
1,2025-07-16,5.5716,5.5722
2,2025-07-17,5.5731,5.5737
3,2025-07-18,5.5460,5.5466
4,2025-07-21,5.5619,5.5625


5. Salva como csv

In [70]:
# Salva como CSV
df.to_csv('cotacao_dolar_ultimos_7_dias.csv', index=False)
print("Arquivo 'cotacao_dolar_ultimos_7_dias.csv' salvo com sucesso!")

Arquivo 'cotacao_dolar_ultimos_7_dias.csv' salvo com sucesso!
